## MATRIX INVERSION AS A QUBO PROBLEM

Ref  
[1.Floating-Point Calculations on a Quantum Annealer:
Division and Matrix Inversion](https://arxiv.org/pdf/1901.06526.pdf)

In this section we present an algorithm for solving a system of linear equations on a
quantum annealer. To precisely define the mathematical problem, let $M$ be a nonsingular $N × N$ real matrix, and let $Y$ be a real $N$ dimensional vector; we then wish to solve the linear equation
$$ M · x = Y$$
The linearity of the system means that there is a unique solution,
$$x = M^{−1} · Y$$

Constructing a quadratic matrix 
$$H(x) = (M x − Y)^2 = (M x − Y)^T · (M x − Y)$$
$$H(x) = x^T M^T Mx - x^T M^T Y - Y^T M x +Y^T Y$$
$$H(x) = \sum_{ijk=1}^{N} M_{ki} M_{kj} x^i x^j - 2\sum_{ij=1}^N Y_j M_{ji} x^i + ||Y^2||$$
To obtain a floating point representation of each component
of $x = (x_1, · · · , x_N)^T$ by expanding in powers of 2 multiplied by Boolean-valued variables
$$\chi^i = \sum_{r=0}^{R-1} 2^{-r}q_r^i $$
$$x^i = 2\chi^i-1$$

And to obtain integer representation, the real value $x_i$

$$ x_i = \sum_{l=-m}^m 2^{-l} q_{i,l}


As before, the domains are given by $\chi^i\in [0, 2)$ and $x^i \in [−1, 3)$, and upon expressing $x$ as a function the $q_r^i$ we can recast $H(x)$ in the form  
$$H(q) = \sum_{i=1}^{N}\sum_{r=0}^{R-1} a_r^i q_r^j + \sum_{i=1}^N \sum_{i\neq j=1}^N \sum_{r=0}^{R-1} \sum_{s=0}^{R-1} b_{rs}^{ij} q_{r}^i q_s^i$$

In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import random, math
import copy

Dimension = 2
qubits = 4
N = Dimension
R = qubits
M = np.array([[1.0, 2.0], [0.5, 0.5]])
Y = np.array([1, 0])

In [15]:
l_max = N*R
print(l_max)
a_l = np.zeros((1, l_max))
QM = np.zeros((qubits*Dimension, qubits*Dimension))
# a_l[0][1] = 2
# print(a_l)
b_lm = np.zeros([l_max, l_max], dtype=float)
# print(b_lm)
for k in range(N):    
    for i in range(N):
        sum_Mkj = 0.0;
        for r in range(R):
            cef1 = 4*pow(2,-2*r)*pow(M[k][i],2)
            sum_kj = 0;
            for j in range(N):
                sum_kj = sum_kj+ M[k][j]
                
            cef2 = 4*pow(2,-r)*M[k][i]*(Y[k]+sum_kj)
            po1 = i*R+r
            a_l[0][po1] = a_l[0][po1] + cef1 - cef2
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
    
# print(a_l)
# print(QM)

for k in range(N):
    for i in range(N):
            for j in range(i,N):
                for r in range(R):
                    for s in range(R):
                        po1 = R*i + r
                        po2 = R*j + s
#                         print("pos1: ",po1," pos2: ", po2)
                        if po1 != po2:
                            qcef = 4*pow(2, -(r+s))*M[k][i]*M[k][j]
                            QM[po1][po2] = QM[po1][po2] + qcef
                        

print(QM)
                                         
# for s in range(R):
# #     i_l = math.floor(l/R)
# #     print("i_m: ", i_l)
#     for r in range(R):
# #         i_m = math.floor(m/R)
# #         print("m: ", m, " i_m: ", i_m)
#         for i in range(N):
#             l = i*R+r
#             m = i*R+s
#             print("s: ",s," r: ",r," i: ",i, " m: ", m, " l: ", l)
#             for k in range(N):
#                 b_lm[l][m] = b_lm[l][m]+4*pow(2,-(r+s)) * M[k][i]*M[k][i]
            
#         b_lm[l][m] = b_lm[l][m] * 4 * pow(2,-(i_l+i_m))

# print(b_lm)

8
[[-13.         2.5        1.25       0.625      9.         4.5
    2.25       1.125   ]
 [  2.5       -7.75       0.625      0.3125     4.5        2.25
    1.125      0.5625  ]
 [  1.25       0.625     -4.1875     0.15625    2.25       1.125
    0.5625     0.28125 ]
 [  0.625      0.3125     0.15625   -2.171875   1.125      0.5625
    0.28125    0.140625]
 [  0.         0.         0.         0.       -17.         8.5
    4.25       2.125   ]
 [  0.         0.         0.         0.         8.5      -12.75
    2.125      1.0625  ]
 [  0.         0.         0.         0.         4.25       2.125
   -7.4375     0.53125 ]
 [  0.         0.         0.         0.         2.125      1.0625
    0.53125   -3.984375]]


## Apply QUBO

In [16]:
qbit_list = []
for val in range(N*R):
    qbit_list.append('q'+str(val+1)) 
print(qbit_list)
Q2 = {}
for i in range(len(qbit_list)):
    for j in range(len(qbit_list)):
        if QM[i][j] != 0:
            Q2[(qbit_list[i],qbit_list[j])] = QM[i][j]
            
print(Q2)
            

['q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8']
{('q1', 'q1'): -13.0, ('q1', 'q2'): 2.5, ('q1', 'q3'): 1.25, ('q1', 'q4'): 0.625, ('q1', 'q5'): 9.0, ('q1', 'q6'): 4.5, ('q1', 'q7'): 2.25, ('q1', 'q8'): 1.125, ('q2', 'q1'): 2.5, ('q2', 'q2'): -7.75, ('q2', 'q3'): 0.625, ('q2', 'q4'): 0.3125, ('q2', 'q5'): 4.5, ('q2', 'q6'): 2.25, ('q2', 'q7'): 1.125, ('q2', 'q8'): 0.5625, ('q3', 'q1'): 1.25, ('q3', 'q2'): 0.625, ('q3', 'q3'): -4.1875, ('q3', 'q4'): 0.15625, ('q3', 'q5'): 2.25, ('q3', 'q6'): 1.125, ('q3', 'q7'): 0.5625, ('q3', 'q8'): 0.28125, ('q4', 'q1'): 0.625, ('q4', 'q2'): 0.3125, ('q4', 'q3'): 0.15625, ('q4', 'q4'): -2.171875, ('q4', 'q5'): 1.125, ('q4', 'q6'): 0.5625, ('q4', 'q7'): 0.28125, ('q4', 'q8'): 0.140625, ('q5', 'q5'): -17.0, ('q5', 'q6'): 8.5, ('q5', 'q7'): 4.25, ('q5', 'q8'): 2.125, ('q6', 'q5'): 8.5, ('q6', 'q6'): -12.75, ('q6', 'q7'): 2.125, ('q6', 'q8'): 1.0625, ('q7', 'q5'): 4.25, ('q7', 'q6'): 2.125, ('q7', 'q7'): -7.4375, ('q7', 'q8'): 0.53125, ('q8', 'q5'): 2.125

In [17]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):26.0, ('q2','q2'):72.0, ('q3','q3'):-6.0, ('q4','q4'):8.0, ('q5','q5'):-13.0, ('q6','q6'):-16.0, ('q7','q7'):23.0, ('q8','q8'):56.0}

quadratic = {('q1','q2'):40.0, ('q1','q5'):2.0, ('q1','q6'):4.0, ('q1','q7'):-2.0, ('q1','q8'):-4.0, ('q2','q5'):4.0, ('q2','q6'):8.0, ('q2','q7'):-4.0, ('q2','q8'):-8.0, ('q3','q4'):40.0, ('q3','q5'):-2.0, ('q3','q6'):-4.0, ('q3','q7'):2.0, ('q3','q8'):4.0, ('q4','q5'):-4.0, ('q4','q6'):-8.0, ('q4','q7'):4.0, ('q4','q8'):8.0, ('q5','q6'):20.0, ('q7','q8'):20.0}

Q = dict(linear)
Q.update(quadratic)

# print(Q)

sampleset = sampler_auto.sample_qubo(Q2, num_reads=1000)
print(sampleset)

   q1 q2 q3 q4 q5 q6 q7 q8     energy num_oc. chain_.
0   1  0  0  1  0  1  1  0 -22.265625      75     0.0
1   1  0  1  0  0  1  0  1 -22.265625      34     0.0
2   1  0  0  1  0  1  0  1 -22.203125      64     0.0
3   1  0  1  0  0  1  1  0   -22.1875      30     0.0
4   1  0  0  0  0  1  1  0   -22.1875      42     0.0
5   1  0  1  1  0  1  0  1 -22.171875      47     0.0
6   1  0  0  0  0  1  0  1 -21.984375      27     0.0
7   1  0  1  1  0  1  1  0 -21.953125      26     0.0
8   0  1  1  1  0  1  1  0 -21.953125      39     0.0
9   1  1  0  0  0  1  0  1 -21.921875      57     0.0
10  1  0  0  0  0  1  1  1 -21.859375      20     0.0
11  1  0  1  1  0  1  0  0 -21.859375      30     0.0
12  1  0  1  0  0  1  0  0   -21.8125      30     0.0
13  1  0  0  1  0  1  1  1 -21.796875      37     0.0
14  0  1  1  1  0  1  1  1 -21.765625      25     0.0
15  1  1  0  0  0  1  0  0     -21.75      41     0.0
16  1  0  0  1  0  1  0  0 -21.609375      19     0.0
17  0  1  1  1  0  1  0  1 -

### Convert Qbit to Decimal 

In [18]:
samples = sampleset.samples()
type(samples[0])
qbit_per_var_count = int(len(qbit_list)/Dimension)
print(qbit_per_var_count)

qbit_per_var = []
for i in range(Dimension):
    qbit_per_var.append(samples[i,qbit_list[qbit_per_var_count*i:qbit_per_var_count*(i+1)]])

print(qbit_per_var)
res = []

for i in range(N):
    res.append(0)
    for j in range(qubits):
        res[i] += (pow(2,-j)*qbit_per_var[i][j])
    res[i] = 2*(res[i]-1)

print("Output: ", res)
print("input A: ", M," b: ", Y)
    


4
[array([1, 0, 0, 1], dtype=int8), array([0, 1, 0, 1], dtype=int8)]
Output:  [0.25, -0.75]
input A:  [[1.  2. ]
 [0.5 0.5]]  b:  [1 0]


In [41]:
import dimod

#print(Q)
J = dimod.qubo_to_ising(Q2)
print(J)

({'q1': -1.0, 'q2': -1.125, 'q3': -0.71875, 'q4': -0.3984375, 'q5': 1.0, 'q6': -0.125, 'q7': -0.21875, 'q8': -0.1484375}, {('q1', 'q2'): 1.25, ('q1', 'q3'): 0.625, ('q1', 'q4'): 0.3125, ('q1', 'q5'): 1.5, ('q1', 'q6'): 0.75, ('q1', 'q7'): 0.375, ('q1', 'q8'): 0.1875, ('q2', 'q3'): 0.3125, ('q2', 'q4'): 0.15625, ('q2', 'q5'): 0.75, ('q2', 'q6'): 0.375, ('q2', 'q7'): 0.1875, ('q2', 'q8'): 0.09375, ('q3', 'q4'): 0.078125, ('q3', 'q5'): 0.375, ('q3', 'q6'): 0.1875, ('q3', 'q7'): 0.09375, ('q3', 'q8'): 0.046875, ('q4', 'q5'): 0.1875, ('q4', 'q6'): 0.09375, ('q4', 'q7'): 0.046875, ('q4', 'q8'): 0.0234375, ('q5', 'q6'): 1.25, ('q5', 'q7'): 0.625, ('q5', 'q8'): 0.3125, ('q6', 'q7'): 0.3125, ('q6', 'q8'): 0.15625, ('q7', 'q8'): 0.078125}, -13.4765625)


In [42]:
sampleset = sampler_auto.sample_ising(J[0], J[1], num_reads=1000)
print(sampleset)

   q1 q2 q3 q4 q5 q6 q7 q8    energy num_oc. chain_.
0  -1 +1 +1 +1 -1 +1 +1 +1 -5.992188     338     0.0
1  -1 +1 +1 +1 -1 +1 +1 -1 -5.492188      57     0.0
2  -1 +1 +1 +1 -1 +1 -1 +1 -4.992188      32     0.0
3  -1 +1 +1 -1 -1 +1 +1 +1 -4.992188      28     0.0
4  +1 -1 +1 +1 -1 +1 +1 +1 -4.992188      83     0.0
5  +1 +1 +1 +1 -1 -1 +1 +1 -4.992188      41     0.0
6  +1 -1 +1 +1 -1 +1 +1 -1 -4.867188      55     0.0
7  +1 +1 +1 -1 -1 -1 +1 +1 -4.867188      34     0.0
8  +1 +1 -1 +1 -1 -1 +1 +1 -4.742188      13     0.0
9  +1 -1 +1 +1 -1 +1 -1 +1 -4.742188      27     0.0
10 +1 +1 -1 +1 -1 +1 -1 +1 -4.617188      16     0.0
11 +1 -1 +1 -1 -1 +1 +1 +1 -4.617188      32     0.0
12 +1 +1 +1 +1 -1 -1 +1 -1 -4.617188      17     0.0
13 +1 +1 +1 -1 -1 +1 -1 +1 -4.554688      15     0.0
14 +1 +1 -1 +1 -1 +1 +1 -1 -4.554688      16     0.0
15 +1 -1 +1 +1 -1 -1 +1 +1 -4.492188      15     0.0
16 +1 +1 -1 +1 -1 +1 +1 +1 -4.492188      10     0.0
17 +1 +1 +1 +1 -1 +1 -1 +1 -4.492188      11  